# Invoke 방법 - Sagemaker + vLLM으로 Deploy한 모델에 Invoke하는 방법들

### 0. 환경 세팅

In [41]:
import io 
import sagemaker
import boto3
import json
from sagemaker import get_execution_role

In [42]:

role = get_execution_role()
region = boto3.Session().region_name
print(role, region)

sagemaker_session = sagemaker.session.Session()
sm_client = boto3.client("sagemaker", region_name=region)
sm_runtime_client = boto3.client("sagemaker-runtime")




arn:aws:iam::396608815783:role/service-role/AmazonSageMaker-ExecutionRole-20250207T102425 us-west-2


### 1. Endpoint name 구하기
- Web Console에서 확인 후 문자열을 복사/붙여 넣기 해 도 됩니다. 
- 그렇지 않은경우, edpoint_name을 검색하여, 최신 endpoint_name을 할당 합니다. 

In [43]:
# endpoint_name = sagemaker.utils.name_from_base("QWEN-2-5-7B-G5")
# endpoint_name = "QWEN-2-5-7B-G5-2025-08-12-03-05-19-368"


all_endpoint = sm_client.list_endpoints()
print(all_endpoint,"\n")
endpoint_name = all_endpoint['Endpoints'][0]['EndpointName']
print(" - endpoint_name : ", endpoint_name)


{'Endpoints': [{'EndpointName': 'gemma3-s3-vllm-async-2025-08-14-02-10-03-005', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:396608815783:endpoint/gemma3-s3-vllm-async-2025-08-14-02-10-03-005', 'CreationTime': datetime.datetime(2025, 8, 14, 2, 10, 5, 750000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2025, 8, 14, 2, 19, 17, 309000, tzinfo=tzlocal()), 'EndpointStatus': 'InService'}, {'EndpointName': 'gemma3-s3-vllm-async-2025-08-14-01-16-07-849', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:396608815783:endpoint/gemma3-s3-vllm-async-2025-08-14-01-16-07-849', 'CreationTime': datetime.datetime(2025, 8, 14, 1, 16, 9, 400000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2025, 8, 14, 1, 25, 31, 472000, tzinfo=tzlocal()), 'EndpointStatus': 'InService'}, {'EndpointName': 'QWEN-2-5-7B-G5-2025-08-12-03-05-19-368', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:396608815783:endpoint/QWEN-2-5-7B-G5-2025-08-12-03-05-19-368', 'CreationTime': datetime.datetime(2025, 8, 12, 3, 5, 

## Invoke 방법 1 : predictor

In [44]:
predictor = sagemaker.Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sagemaker_session,
    serializer=sagemaker.serializers.JSONSerializer(),
    deserializer=sagemaker.deserializers.JSONDeserializer(),
)


In [46]:
# 호출 예시
response = predictor.predict({
    # "inputs": "AI Agent에 대해 100단어 내외로 설명해 주세요.",
    # {"type": "image", "url": "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/p-blog/candy.JPG"},
    "inputs": "AI Agent에 대해서 설명 해 주세요.",
    "parameters": {
        "max_new_tokens": 1024,
        "stop": ["<|endoftext|>"]  # Stop sequences 지정 가능
    }
})

print(response['generated_text'])



AI Agent는 간단히 말해, 목표를 달성하기 위해 스스로 생각하고 행동할 수 있는 인공지능 시스템입니다. 마치 사람처럼 환경을 인식하고, 계획을 세우고, 판단하고, 행동하여 원하는 결과를 얻어내는 것이죠. 

**좀 더 자세히 설명하자면:**

* **기능 구성 요소:** AI Agent는 일반적으로 다음과 같은 핵심 구성 요소로 이루어져 있습니다.
    * **센서 (Sensors):** 환경에서 정보를 수집하는 역할을 합니다. (예: 카메라, 마이크, 온도 센서 등)
    * **계획자 (Planner):** 수집된 정보를 바탕으로 목표를 달성하기 위한 계획을 세웁니다.
    * **행동자 (Actuator):** 계획에 따라 환경에 영향을 미치는 행동을 수행합니다. (예: 로봇 팔, 자동차, 음성 출력 등)
    * **기억 (Memory):** 과거 경험과 정보를 저장하고 활용하여 학습하고 개선합니다.
* **작동 방식:** AI Agent는 다음과 같은 과정을 반복적으로 수행합니다.
    1. **관찰 (Observe):** 센서를 통해 환경을 관찰하고 현재 상태를 파악합니다.
    2. **생각 (Think):** 계획자를 사용하여 현재 상태와 목표를 고려하여 최적의 행동을 결정합니다.
    3. **행동 (Act):** 행동자를 통해 결정된 행동을 실행합니다.
    4. **학습 (Learn):**  행동의 결과와 환경의 변화를 통해 기억에 저장하고, 다음에 더 나은 행동을 선택하도록 학습합니다.
* **다양한 유형의 AI Agent:** AI Agent는 다양한 분야와 목적에 따라 다양한 유형으로 분류될 수 있습니다.
    * **로봇:** 물리적인 환경에서 작업을 수행하는 AI Agent (예: 청소 로봇, 제조 로봇)
    * **가상 비서:** 음성 또는 텍스트를 통해 사용자의 명령을 처리하고 정보를 제공하는 AI Agent (예: Siri, Alexa)
    * **게임 AI:** 게임 내 캐릭터의 행동을 제어하

## Invoke 방법 2: Invoke EndPoint
- Predictor를 사용하지 않고, Endpoint_name만을 사용하는 경우
- Boto3를 사용한 Low-Level AWS SDK 호출

In [10]:
# Endpoint_name 확인
endpoint_name

'gemma3-s3-vllm-async-2025-08-13-11-02-49-152'

In [16]:
import boto3
import json

# SageMaker runtime client 생성
runtime = boto3.client('sagemaker-runtime', region_name=region)

# endpoint_name = 'qwen-2-5-endpoint'
# endpoint_name = "QWEN-2-5-7B-G5-2025-08-12-03-05-19-368"

payload = {
    "inputs": "AI Agent에 대해서 설명 해 주세요.",
    "parameters": {
        "max_new_tokens": 512,
        "stop": ["<|endoftext|>"]
    }
}

response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='application/json',
    Body=json.dumps(payload)
)

result = response['Body'].read().decode('utf-8')
print(json.loads(result)['generated_text'])




AI 에이전트(AI agent)는 주어진 환경에서 특정 목표를 달성하기 위해 스스로 판단하고 행동하는 인공지능 시스템입니다. 인간과 마찬가지로, AI 에이전트도 감각 기관을 통해 환경을 인식하고, 지능을 이용하여 정보를 처리하고, 행동을 선택하고, 실행하여 결과를 평가하고, 이를 통해 학습하여 성능을 개선하는 과정을 반복합니다.

**AI 에이전트의 구성 요소**

1.  **감각 기관 (Sensors):** 환경에서 정보를 수집하는 장치입니다. 예를 들어, 로봇의 카메라, 마이크, 센서 등이 감각 기관에 해당합니다.
2.  **환경 (Environment):** 에이전트가 상호작용하는 공간입니다. 현실 세계의 물리적인 환경일 수도 있고, 가상 환경일 수도 있습니다.
3.  **행동 결정 모듈 (Decision-Making Module):** 감각 기관을 통해 수집된 정보를 바탕으로 어떤 행동을 취할지 결정하는 부분입니다. 여기에는 학습된 모델, 규칙 기반 시스템, 또는 기타 지능적인 알고리즘이 사용될 수 있습니다.
4.  **행동 실행 모듈 (Action Execution Module):** 결정된 행동을 실제로 수행하는 부분입니다. 예를 들어, 로봇 팔의 움직임, 웹 페이지의 클릭, 텍스트 메시지의 전송 등이 행동 실행에 해당합니다.
5.  **학습 모듈 (Learning Module):** 에이전트의 행동 결과를 평가하고, 이를 바탕으로 자신의 행동 전략을 개선하는 부분입니다. 강화 학습, 지도 학습, 비지도 학습 등 다양한 학습 방법을 사용할 수 있습니다.

**AI 에이전트의 종류**

*   **단순 반응형 에이전트:** 미리 정의된 규칙에 따라 행동합니다.
*   **목표 지향 에이전트:** 주어진 목표를 달성하기 위해 행동합니다.
*   **학습 에이전트:** 경험을 통해 스스로 학습하고 성능을 개선합니다.
*   **협력 에이전트:** 다른 에이전트와 협력하여 목표를 달성합니다.
*   **경쟁 에이전트:** 다른 에이전트와 경쟁하여 목표를

### Invoke 방법 3 : Streaming API를 이용한 호출 (Streaming 응답 지원 시)

In [17]:
res = sm_runtime_client.invoke_endpoint_with_response_stream(
    EndpointName=endpoint_name,
    Body=json.dumps(payload),
    ContentType='application/json',
)

buffer = ""
string = ""

for event in res["Body"]:
    # 각 chunk 처리
    # print(event)
    if 'PayloadPart' in event:
        chunk = event.get("PayloadPart").get("Bytes").decode()
        buffer += chunk

    print(chunk, flush=True)


{"generated_text": "\n\nAI 에이전트 (AI Agent)는 특정 목표를 달성하기 위해 스스로 학습하고, 의사 결정을 내리고, 행동하는 능력을 갖춘 인공지능 시스템입니다. 인간과 유사한 방식으로 환경과 상호 작용하며, 복잡한 작업을 수행할 수 있도록 설계되었습니다. \n\n**AI 에이전트의 주요 특징:**\n\n*   **환경 인식 (Perception):** 센서나 다른 입력 채널을 통해 환경에 대한 정보를 수집합니다. (예: 카메라, 마이크, API 등)\n*   **의사 결정 (Decision-Making):** 수집된 정보를 바탕으로 어떤 행동을 취할지 결정합니다. (예: 강화 학습, 지도 학습, 규칙 기반 시스템 등)\n*   **행동 (Action):** 결정된 행동을 실행합니다. (예: 로봇 팔 움직이기, 웹 페이지에 입력하기, 이메일 보내기 등)\n*   **학습 (Learning):** 경험을 통해 자신의 행동을 개선하고, 더 나은 의사 결정을 내릴 수 있도록 스스로 학습합니다. (예: 강화 학습, 지도 학습 등)\n\n**AI 에이전트의 종류:**\n\n*   **강화 학습 에이전트 (Reinforcement Learning Agent):** 보상을 최대화하는 방향으로 행동을 학습합니다. (예: 게임 AI, 로봇 제어)\n*   **지도 학습 에이전트 (Supervised Learning Agent):** 주어진 데이터에 대한 정답을 학습합니다. (예: 이미지 인식, 자연어 처리)\n*   **반지도 학습 에이전트 (Semi-Supervised Learning Agent):** 레이블이 지정된 데이터와 레이블이 지정되지 않은 데이터를 모두 사용하여 학습합니다.\n*   **규칙 기반 에이전트 (Rule-Based Agent):** 미리 정의된 규칙에 따라 행동을 결정합니다. (예: 챗봇, 자동화 시스템)\n\n**AI 에이전트의 예시:**\n\n*   **챗봇 (Chatbot):** 사용자의 질문에 답변하고, 정보를 제공합

### 4. REST API 직접 호출

In [18]:
import requests
import boto3
import json
from botocore.auth import SigV4Auth
from botocore.awsrequest import AWSRequest

In [19]:
url = f"https://runtime.sagemaker.{region}.amazonaws.com/endpoints/{endpoint_name}/invocations"
session = boto3.Session()
credentials = session.get_credentials()

# 요청 생성
request = AWSRequest(
    method='POST',
    url=url,
    data=json.dumps(payload),
    headers={'Content-Type': 'application/json'}
)

# AWS 서명 추가
SigV4Auth(credentials, 'sagemaker', region).add_auth(request)

# 요청 실행
response = requests.post(
    url,
    data=request.body,
    headers=dict(request.headers)
)

response.json()

{'generated_text': '\n\nAI Agent는 인공지능 기술을 활용하여 특정 목표를 달성하기 위해 스스로 판단하고 행동하는 시스템입니다. 인간과 유사한 방식으로 환경을 인식하고, 정보를 처리하고, 의사 결정을 내렸다가, 행동을 수행하며, 그 결과를 평가하고 다시 행동을 조정하는 과정을 반복합니다.\n\n**AI Agent의 구성 요소**\n\n*   **환경 (Environment):** AI Agent가 상호 작용하는 외부 세계입니다. 예: 웹사이트, 게임, 로봇의 물리적 환경 등\n*   **감지기 (Sensors):** 환경에서 정보를 수집하는 장치입니다. 예: 카메라, 마이크, 센서 등을 통해 환경 데이터를 수집\n*   **행동기 (Actuators):** 환경에 영향을 미치는 장치입니다. 예: 로봇 팔, 웹사이트에 댓글 작성, 게임 캐릭터 이동 등\n*   **지능 모듈 (Intelligence Module):** AI Agent의 핵심 부분으로, 다음을 수행합니다.\n    *   **인지 (Perception):** 감지기를 통해 얻은 정보를 해석하고 이해합니다.\n    *   **계획 (Planning):** 목표를 달성하기 위한 행동 계획을 수립합니다.\n    *   **의사 결정 (Decision-Making):** 다양한 행동 옵션 중에서 최적의 행동을 선택합니다.\n    *   **학습 (Learning):** 경험을 통해 자신의 행동을 개선합니다.\n\n**AI Agent의 종류**\n\n*   **단순 AI Agent:** 미리 정의된 규칙에 따라 작동합니다.\n*   **복잡 AI Agent:** 머신러닝 기술을 사용하여 스스로 학습하고 적응합니다.\n    *   **강화 학습 Agent:** 보상을 통해 행동을 학습합니다. (예: 게임 AI, 로봇 제어)\n    *   **지도 학습 Agent:** 학습 데이터를 통해 특정 작업을 수행하도록 학습합니다. (예: 이미지 인식, 자연어 처리)\n    *   *